# ローカル環境からH2O AI Cloudへのアクセス

PythonからAI Cloud上へのDriverlessAI、MLOpsへのアクセス：https://docs.h2o.ai/haic-documentation/guide/general/using-platform-token  
DriverlessAIに対するPython操作詳細：https://docs.h2o.ai/driverless-ai/1-10-lts/docs/userguide/python_client.html  
MLOpsに対するPython操作詳細：https://docs.h2o.ai/mlops/py-client/overview

<img src="image/AiCloud_ApiAccess.png" width=600px>

In [1]:
!python -V

Python 3.8.16


必要Pythonライブラリ
- **h2o_authn** 
- **h2o_mlops_client** - MLOpsクライアント
- **h2osteam** - SteamによりDriverless AIへアクセスする場合
- **h2o-engine-manager** - AI Engine ManagerによりDriverless AIへアクセスする場合

In [2]:
!pip list | grep h2o

h2o                                                           3.40.0.4
h2o-authn                                                     1.0.0
h2o-cloud-discovery                                           1.0.1
h2o-engine-manager                                            0.4.0
h2o-mlops-client                                              0.56.1+dd66f93.rel0.56.1.2
h2osteam                                                      1.8.12
scoring-h2oai-experiment-1fdfee78-3dec-11eb-8f87-0242ac110002 1.0.0


In [3]:
import sys
sys.path.append('utility/')    # h2o_ai_cloud.pyはutilityディレクトリ上に配置済み

In [4]:
'''
from h2o_ai_cloud import token_provider, steam_client
import h2osteam 
import h2o_engine_manager
'''

In [17]:
import h2o_authn 
import getpass 

import h2osteam 
import h2o_engine_manager 


In [9]:
## トークンによる認証

print(f"Visit https://internal.dedicated.h2o.ai/auth/get-platform-token to get your platform token") 

token_provider = h2o_authn.TokenProvider(
    refresh_token=getpass.getpass('Enter your platform token: '),
    client_id="hac-platform-public", 
    token_endpoint_url="https://auth.internal.dedicated.h2o.ai/auth/realms/hac/protocol/openid-connect/token" 
)

Visit https://internal.dedicated.h2o.ai/auth/get-platform-token to get your platform token
Enter your platform token: ········


In [38]:
# token_provider()を実行すると登録したトークンが表示される
#token_provider()

## Steam

In [25]:
## Steamへのログイン

h2osteam.login(
    url="https://steam.internal.dedicated.h2o.ai", 
    access_token=token_provider()
)
# -> Steamアカウントが削除されている

HTTPError: '401: This user has been deactivated\n'

## AI Engine Manager

In [80]:
## Engine Managerへのログイン

aiem = h2o_engine_manager.login(
    environment="https://internal.dedicated.h2o.ai", 
    token_provider=token_provider
)
aiem

### ドキュメント
[AI Engine Python API](https://h2oai.github.io/ai-engine-manager/docs/py/clients/)

In [88]:
aiem.dai_engine_client.list_engines()

{'engines': [{'annotations': {},
 'api_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/snk',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7ff13a2bed00>,
 'config': {},
 'cpu': 8,
 'create_time': datetime.datetime(2023, 6, 3, 22, 44, 11, tzinfo=tzutc()),
 'creator': 'users/bc34ef4d-ff29-4375-8bab-04d860fcac32',
 'creator_display_name': 'yuki.shimada@h2o.ai',
 'current_idle_duration': None,
 'current_running_duration': None,
 'delete_time': None,
 'display_name': 'snk',
 'engine_id': 'snk',
 'gpu': 0,
 'login_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/snk/oidc/login',
 'max_idle_duration': '1h',
 'max_running_duration': '8h',
 'memory_bytes': '32Gi',
 'name': 'workspaces/default/daiEngines/snk',
 'reconciling': False,
 'resume_time': datetime.datetime(2023, 6, 7, 6, 0, 40, tzinfo=tzutc()),
 'state': <DAIEngineState.STATE_PAUSED: 'STATE_PAUSED'>,
 'storage_bytes': '6

In [89]:
# DAIインスタンスの取得
dai_access = aiem.dai_engine_client.get_engine(engine_id='new-dai-engine-8052')
dai_access

{'annotations': {},
 'api_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-8052',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7ff13a2bed00>,
 'config': {},
 'cpu': 8,
 'create_time': datetime.datetime(2023, 6, 1, 2, 36, 30, tzinfo=tzutc()),
 'creator': 'users/bc34ef4d-ff29-4375-8bab-04d860fcac32',
 'creator_display_name': 'yuki.shimada@h2o.ai',
 'current_idle_duration': None,
 'current_running_duration': None,
 'delete_time': None,
 'display_name': 'Demo-20230601',
 'engine_id': 'new-dai-engine-8052',
 'gpu': 0,
 'login_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-8052/oidc/login',
 'max_idle_duration': '2h',
 'max_running_duration': '8h',
 'memory_bytes': '32Gi',
 'name': 'workspaces/default/daiEngines/new-dai-engine-8052',
 'reconciling': False,
 'resume_time': datetime.datetime(2023, 6, 11, 1, 30, 55, tzinfo=tzutc()),
 'state': <DAIE

In [59]:
# インスタンス状況確認
dai_access.state

<DAIEngineState.STATE_PAUSED: 'STATE_PAUSED'>

In [60]:
# インスタンスの起動
dai_access.resume()

In [63]:
dai_access.state

<DAIEngineState.STATE_STARTING: 'STATE_STARTING'>

In [71]:
dai = dai_access.connect()
type(dai)

driverlessai._core.Client

In [76]:
dai.datasets.list()

    | Type    | Key                                  | Name
----+---------+--------------------------------------+----------------------
  0 | Dataset | 0f74887e-0026-11ee-ab62-1235030f5bf3 | UCI_Credit_Card3.csv
  1 | Dataset | 040a3e3e-0026-11ee-ab62-1235030f5bf3 | TitanicData2.csv
  2 | Dataset | 0352ed60-0026-11ee-ab62-1235030f5bf3 | BostonHousing.csv

In [75]:
dai.experiments.list()

    | Type       | Key                                  | Name
----+------------+--------------------------------------+---------
  0 | Experiment | 3b610574-0027-11ee-ab62-1235030f5bf3 | boston1

### References
(Internal) [1 End to End Demo.ipynb](https://github.com/h2oai/jupyterlabs-app/blob/96f42b40977350f6ee5cd139467314e8beb0721e/Example%20Notebooks/1%20End%20to%20End%20Demo.ipynb)